In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import hashlib
from skimage import exposure
from typing import List, Tuple, Optional

# Importa as funções do SAHI para sliced inference
from sahi.predict import get_sliced_prediction
from sahi import AutoDetectionModel

# Constante para o tamanho da janela (utilizada como dimensão do slice)
WINDOW_SIZE = 254

class ImageProcessor:
    def __init__(self, model_path: str, window_size: int = WINDOW_SIZE) -> None:
        """
        Inicializa o processador de imagem.

        :param model_path: Caminho para o modelo de detecção de objetos.
        :param window_size: Tamanho da janela (slice) para o processamento.
        """
        self.window_size: int = window_size
        self.model_path: str = model_path
        self.image: Optional[np.ndarray] = None
        self.processed_image: Optional[np.ndarray] = None
        self.total_objects: int = 0

    def load_image(self, image_path: str) -> np.ndarray:
        """
        Carrega uma imagem do disco e converte de BGR para RGB.

        :param image_path: Caminho para o arquivo de imagem.
        :return: Imagem em formato RGB.
        :raises ValueError: Se a imagem não for carregada.
        """
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    def normalize_image(self, image: np.ndarray) -> np.ndarray:
        """
        Ajusta o contraste da imagem usando correção gama.

        :param image: Imagem a ser normalizada.
        :return: Imagem com contraste ajustado.
        """
        return exposure.adjust_gamma(image, gamma=1.5)

    def apply_sahi_pipeline(self) -> None:
        """
        Processa a imagem carregada utilizando o SAHI para sliced inference.
        O SAHI fatiará a imagem, realizará a detecção em cada slice e unirá os resultados.
        """
        if self.image is None:
            raise ValueError("Nenhuma imagem foi carregada para processamento.")

        # Pré-processa a imagem (ex.: normalização)
        processed_input = self.normalize_image(self.image)

        # Instancia o modelo de detecção utilizando o AutoDetectionModel do SAHI
        detection_model = AutoDetectionModel.from_pretrained(
            model_type="ultralytics",
            model_path=self.model_path,
            confidence_threshold=0.3,
            device="cpu"  # ou 'cuda:0'
        )

        # Executa a predição fatiada
        result = get_sliced_prediction(
            image=processed_input,
            detection_model=detection_model,
            slice_height=self.window_size,
            slice_width=self.window_size,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2
        ) 
        print(f"Detecções realizadas: {len(result.object_prediction_list)}")

        # Exporta os visuais para um diretório temporário
        temp_folder = "temp_visuals"
        os.makedirs(temp_folder, exist_ok=True)
        result.export_visuals(export_dir=temp_folder)
        annotated_img_path = os.path.join(temp_folder, "prediction_visual.png")
        
        # Carrega a imagem anotada (convertendo para RGB)
        annotated = cv2.imread(annotated_img_path)
        if annotated is None:
            raise ValueError("Não foi possível carregar a imagem anotada exportada pelo SAHI.")
        self.processed_image = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        self.total_objects = len(result.object_prediction_list)

    def show_results(self) -> None:
        """
        Exibe a imagem original e a imagem final processada (anotada com as detecções).
        """
        if self.image is None or self.processed_image is None:
            raise ValueError("Imagens não disponíveis para exibição.")
        fig, axes = plt.subplots(2, 1, figsize=(20, 15))
        axes[0].imshow(self.image)
        axes[0].set_title("Imagem Original")
        axes[0].axis("off")
        axes[1].imshow(self.processed_image)
        axes[1].set_title(f"Imagem Processada Final\nObjetos: {self.total_objects}")
        axes[1].axis("off")
        plt.tight_layout()
        plt.show()

    def process_folder(self, folder_path: str) -> None:
        """
        Processa todas as imagens de uma pasta, salva os resultados e gera um relatório.

        :param folder_path: Caminho para a pasta que contém as imagens.
        """
        report: List[Tuple[str, int]] = []
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        for filename in image_files:
            full_path = os.path.join(folder_path, filename)
            try:
                self.image = self.load_image(full_path)
            except ValueError as e:
                print(e)
                continue

            self.apply_sahi_pipeline()
            report.append((filename, self.total_objects))
            print(f"Processado {filename}: {self.total_objects} objetos encontrados.")

            # Salva a imagem processada em uma subpasta "processed"
            processed_output_folder = os.path.join(folder_path, "processed")
            os.makedirs(processed_output_folder, exist_ok=True)
            output_image_path = os.path.join(processed_output_folder, f"processed_{filename}")
            # self.processed_image já é um numpy array em RGB
            cv2.imwrite(output_image_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))

        # Gera o relatório e salva em um arquivo txt
        report_path = os.path.join(folder_path, "report.txt")
        with open(report_path, "w") as f:
            f.write("+===================================+\n")
            f.write("Filename - Objetos\n")
            f.write(f"Modelo: {os.path.basename(self.model_path)}\n")
            f.write("+===================================+\n")
            for filename, count in report:
                f.write(f"{filename} - {count}\n")
            f.write("+===================================+\n")
        print(f"Relatório salvo em {report_path}")

def main() -> None:
    # Defina os caminhos para o modelo e para a pasta de imagens
    model_path = '/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/draft-actual/best-v1.pt'
    folder_path = '/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/base-4'
    
    processor = ImageProcessor(model_path=model_path)
    processor.process_folder(folder_path)
    # Para visualizar os resultados individualmente, descomente a linha abaixo:
    # processor.show_results()

if __name__ == "__main__":
    main()


Performing prediction on 330 slices.
Detecções realizadas: 43
Processado paleta2_60.png: 43 objetos encontrados.
Performing prediction on 396 slices.
Detecções realizadas: 61
Processado arm 43 108 ovos.jpg: 61 objetos encontrados.
Performing prediction on 320 slices.
Detecções realizadas: 11
Processado paleta4_20.png: 11 objetos encontrados.
Performing prediction on 352 slices.
Detecções realizadas: 24
Processado paleta7_32.png: 24 objetos encontrados.
Performing prediction on 330 slices.
Detecções realizadas: 11
Processado paleta8_0.png: 11 objetos encontrados.
Performing prediction on 363 slices.
Detecções realizadas: 10
Processado paleta6_12.png: 10 objetos encontrados.
Performing prediction on 363 slices.
Detecções realizadas: 35
Processado paleta1_41.png: 35 objetos encontrados.
Performing prediction on 352 slices.
Detecções realizadas: 19
Processado paleta5_29.png: 19 objetos encontrados.
Performing prediction on 330 slices.
Detecções realizadas: 41
Processado paleta3_18.png: 41 